# Classify the Galaxies

## Set Up

In [1]:
#import all necessary libraries
import os

from pyspark.sql.functions import lit
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import IPython.display as dp
from pyspark.ml.image import ImageSchema
from sparkdl.image import imageIO

spark = SparkSession \
    .builder \
    .master("local") \
    .appName("review_and_category_analytics") \
    .config("spark.executor.memory", '8g') \
    .config('spark.executor.cores', '4') \
    .config('spark.cores.max', '4') \
    .config("spark.driver.memory",'8g') \
    .getOrCreate()

sc = spark.sparkContext

sqlCtx = SQLContext(sc)

Using TensorFlow backend.


### Create DataFrame of Images

In [ ]:
fs = os.listdir("data/galaxy_images_classified/other") #get list of image file names
fs.sort() #sort by file name

images = [] #create blank list which will store image objects

for ea in fs: #for each image in the list of images
    #add images object to the list of images
    images.append(dp.Image(filename="data/images_training_rev1/" + ea, format="png"))

In [ ]:
#Show first 5 images
for ea in range(5):
    dp.display_png(images[ea])

#### IMPORTANT!!!! Change directory to full subset when not testing

In [ ]:
#create images dataframe

#image_df = ImageSchema.readImages("data/images_training_rev1/")

image_fd = ImageSchema.readImages("data/subset_images/")

In [ ]:
image_df.show(5)